In [8]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def current_stock_price(ticker):
    url = f"https://eodhd.com/api/real-time/{ticker}.US?api_token={API_KEY}&fmt=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return float(data.get("close", 0))  # ✅ Only return float
    except Exception as e:
        logging.warning(f"Failed to fetch current price for {ticker}: {e}")
        return 0

def fetch_filtered_options(symbol, exp_from, exp_to, strike_from, strike_to):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[type]": "call",
        "filter[exp_date_from]": exp_from,
        "filter[exp_date_to]": exp_to,
        "filter[strike_from]": strike_from,
        "filter[strike_to]": strike_to,
        "sort": "exp_date",
        "page[limit]": 1000
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        return response.json().get("data", [])
    except Exception as e:
        logging.warning(f"Error fetching options contracts for {symbol}: {e}")
        return []

def analyze_options_unicorn(
    tickers,
    option_type="call",
    days_until_exp=90,
    strike_pct=0.2,
    days_to_gain=30,
    stock_gain_pct=0.1,
    risk_free_rate=0.05
):
    today = datetime.now(timezone.utc)
    eval_date = today + timedelta(days=days_to_gain)
    exp_from = (today + timedelta(days=days_until_exp - 30)).strftime("%Y-%m-%d")
    exp_to = (today + timedelta(days=days_until_exp + 30)).strftime("%Y-%m-%d")

    results = []

    for ticker in tickers:
        try:
            current_price = current_stock_price(ticker)
            if current_price <= 0:
                logging.info(f"Skipping {ticker}: invalid current price.")
                continue

            target_strike = current_price * (1 + strike_pct)
            lower = target_strike * 0.95
            upper = target_strike * 1.05

            options_data = fetch_filtered_options(ticker, exp_from, exp_to, lower, upper)
            if not options_data:
                logging.info(f"No options data returned for {ticker}")
                continue

            for opt in options_data:
                attr = opt.get("attributes", {})
                strike = attr.get("strike")
                exp_date_str = attr.get("exp_date")
                last_price = attr.get("last", 0)
                iv = attr.get("volatility", 0.3)

                # Extract Greeks from the API
                delta = attr.get("delta")
                theta = attr.get("theta")

                if not strike or not exp_date_str:
                    continue

                exp_date = datetime.strptime(exp_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
                exp_days = (exp_date - today).days
                T_eval = max((exp_date - eval_date).days / 365, 0.0001)

                sim_stock = current_price * (1 + stock_gain_pct)
                est_value = black_scholes_call_price(sim_stock, strike, T_eval, risk_free_rate, iv)

                result = {
                    "Ticker": ticker,
                    "Expiration": exp_date_str,
                    "Strike": strike,
                    "% OTM/ITM": round((strike - current_price) / current_price * 100, 2),
                    "Current Stock": round(current_price, 2),
                    "Simulated Stock": round(sim_stock, 2),
                    "Current Premium": round(last_price, 2),
                    "Simulated Premium": round(est_value, 2),
                    "Days Until Expiration": exp_days,
                    "Days to Gain": days_to_gain,
                    "Stock Gain %": round(stock_gain_pct * 100, 2),
                    "% Gain": round(((est_value - last_price) / last_price) * 100, 2) if last_price else "NA",
                    "Implied Volatility": round(iv * 100, 2),
                    "Delta": round(delta, 4) if delta is not None else "NA",
                    "Theta": round(theta, 4) if theta is not None else "NA"
                }
                results.append(result)

        except Exception as e:
            logging.error(f"Error processing {ticker}: {e}")

    return pd.DataFrame(results)

# Helper function to run multiple parameter sets
def run_multiple_analyses(param_sets):
    combined_df = pd.DataFrame()
    for params in param_sets:
        label = params.pop("label", "")
        df = analyze_options_unicorn(**params)
        df["Run_Label"] = label
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

# # # Run script
# # if __name__ == "__main__":
# param_sets = [
#     {
#         "tickers": ["AAPL", "MSFT"],
#         "option_type": "call",
#         "days_until_exp": 60,
#         "strike_pct": 0.1,
#         "days_to_gain": 20,
#         "stock_gain_pct": 0.05,
#         "label": "Short-Term Conservative"
#     },
#     {
#         "tickers": ["AAPL", "AMD"],
#         "option_type": "call",
#         "days_until_exp": 120,
#         "strike_pct": 0.3,
#         "days_to_gain": 45,
#         "stock_gain_pct": 0.15,
#         "label": "Long-Term Aggressive"
#     }
# ]

# combined_results = run_multiple_analyses(param_sets)
# display(combined_results)
#     # combined_results.to_csv("combined_unicorn_options.csv", index=False)


,Ticker,Expiration,Strike,% OTM/ITM,Current Stock,Simulated Stock,Current Premium,Simulated Premium,Days Until Expiration,Days to Gain,Stock Gain %,% Gain,Implied Volatility,Delta,Theta,Run_Label
0,AAPL,2025-06-20,217.5,5.14,206.86,217.20,2.72,3.59,29,20,5.0,31.88,26.47,0.2846,-0.0932,Short-Term Conservative
1,AAPL,2025-06-20,220.0,6.35,206.86,217.20,2.10,2.48,29,20,5.0,17.92,26.30,0.2349,-0.0834,Short-Term Conservative
2,AAPL,2025-06-20,222.5,7.56,206.86,217.20,1.59,1.63,29,20,5.0,2.23,26.10,0.1901,-0.0729,Short-Term Conservative
3,AAPL,2025-06-20,225.0,8.77,206.86,217.20,1.22,1.04,29,20,5.0,-14.92,26.17,0.1535,-0.0635,Short-Term Conservative
4,AAPL,2025-06-20,227.5,9.98,206.86,217.20,0.93,0.64,29,20,5.0,-31.53,26.26,0.1227,-0.0545,Short-Term Conservative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,AAPL,2025-10-17,280.0,35.36,206.86,237.89,0.73,2.67,148,45,15.0,266.27,26.46,0.0541,-0.0139,Long-Term Aggressive
70,AMD,2025-09-19,145.0,27.74,113.51,130.54,3.22,5.61,120,45,15.0,74.21,44.06,0.2188,-0.0376,Long-Term Aggressive
71,AMD,2025-09-19,150.0,32.15,113.51,130.54,2.65,4.47,120,45,15.0,68.55,44.64,0.1856,-0.0343,Long-Term Aggressive
72,AMD,2025-10-17,145.0,27.74,113.51,130.54,4.15,7.33,148,45,15.0,76.71,43.55,0.2505,-0.0365,Long-Term Aggressive


In [9]:
import requests
import json

# API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

def fetch_specific_option(symbol, strike, exp_date, option_type="call"):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[strike_eq]": strike,
        "filter[exp_date_eq]": exp_date,
        "filter[type]": option_type,
        "page[limit]": 1
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json().get("data", [])
        if not data:
            print(f"No option found for {symbol} {strike} {option_type.upper()} expiring {exp_date}")
            return

        contract = data[0].get("attributes", {})
        print(json.dumps(contract, indent=4))  # Pretty-print the full contract data

    except Exception as e:
        print(f"Error fetching option: {e}")

# Example usage
if __name__ == "__main__":
    fetch_specific_option(
        symbol="AAPL",
        strike=225,
        exp_date="2025-07-18",
        option_type="call"
    )


{
    "contract": "AAPL250718C00225000",
    "underlying_symbol": "AAPL",
    "exp_date": "2025-07-18",
    "expiration_type": "monthly",
    "type": "call",
    "strike": 225,
    "exchange": "NASDAQ",
    "currency": "USD",
    "open": 3,
    "high": 3.25,
    "low": 2.65,
    "last": 2.92,
    "last_size": 1,
    "change": -0.33,
    "pctchange": -10.15,
    "previous": 3.25,
    "previous_date": "2025-05-19",
    "bid": 2.88,
    "bid_date": "2025-05-20 19:59:59",
    "bid_size": 22,
    "ask": 2.92,
    "ask_date": "2025-05-20 19:59:59",
    "ask_size": 91,
    "moneyness": -0.08999999999999997,
    "volume": 1795,
    "volume_change": 303,
    "volume_pctchange": 20.31,
    "open_interest": 14208,
    "open_interest_change": 116,
    "open_interest_pctchange": 0.82,
    "volatility": 0.258,
    "volatility_change": 0.0082,
    "volatility_pctchange": 3.27,
    "theoretical": 2.92,
    "delta": 0.242255,
    "gamma": 0.014559,
    "theta": -0.061749,
    "vega": 0.259763,
    "rho